In [1]:
import mailbox
import email.utils
import pandas as pd
import exifread
from datetime import datetime, timezone
import pytz
from ipywidgets import IntProgress, IntText

In [2]:
filename = "tgthorley.mbox"
filepath = './data/test/'
mbox = mailbox.mbox(filepath+ filename)

In [3]:
#takes an image path and retruns exif data
def parse_img(imgpath):
    try:
        f = open(imgpath, 'rb')
        tags = exifread.process_file(f)
        return tags
    except:
        print("Failed to extract exif %s" % imgpath)
        return []

In [4]:
#takes a message with an attached immage  and returns the exif data
def getexif_from_attached_image(message):
    for part in message.walk():
        if 'image' in part.get_content_type():
            try:
                filename = part.get_filename()
                filepath = "./attachements/"+filename
                fb = open(filepath,'wb')
                fb.write(part.get_payload(decode=True))
                fb.close()
                tags = parse_img(filepath)
                return tags
            except:
                print ("File not found", filename)
                continue
        else: continue

In [5]:
emails = []
i=0
progressBar = IntProgress(description='Processing emails '+str(len(mbox)), min=0, max=len(mbox), style = {'description_width': 'initial'})
progressText = IntText(value=0, description='Processed so far: ', style = {'description_width': 'initial'})
display(progressBar, progressText)

for message in mbox:
    i= i+1
    if message.get_content_maintype() == 'multipart':
        for part in message.walk():
            if 'image' in part.get_content_type():
                tags = getexif_from_attached_image(message)
                try:
                    if "EXIF DateTimeOriginal" in tags:
                        emaildate = email.utils.parsedate_to_datetime(message['Date']).astimezone(pytz.utc)
                        imgdate = datetime.strptime(tags["EXIF DateTimeOriginal"].values, '%Y:%m:%d %H:%M:%S').astimezone(pytz.utc)
                        timediff = emaildate - imgdate
                        if timediff.total_seconds() > 0 and timediff.total_seconds() < 86400:
                            print ('WARNING: Image taken less than a day after sending\n', message['Message-ID'], message['Subject'] )
                        email_sum = {
                            "email_date": emaildate,
                            "img_date": imgdate,
                            "_id": message['Message-ID'],
                            "subject": message['Subject'],
                            "timediff": timediff
                        }
                        emails.append(email_sum)
                        progressBar.value = i
                        progressText.value = i
                except: 
                    print("Error", message['Subject'])
                    continue

IntProgress(value=0, description='Processing emails 2364', max=2364, style=ProgressStyle(description_width='in…

IntText(value=0, description='Processed so far: ', style=DescriptionStyle(description_width='initial'))

 <CAPRuAkYabyKs-W4n1w+-7hYhDGM5qn=2VwUaNOUDTe81VEiHfQ@mail.gmail.com> nudies
 <CAPRuAkbb_-=Cx7t6=s-mt1_RqHQoE8o6pYRFEOrLU--BnVtDig@mail.gmail.com> 
 <CAPRuAkahivFm9Jwct-1nQWTWaTSoRghT9qWan9mW=Orcmtm8JA@mail.gmail.com> Fwd: Best B'day Present Ever
 <CABUDHgLp5sCYrzJG_DLaoypdDvG=zunXjK+uFddmviZRgFWsBg@mail.gmail.com> Happy one month, Julian James!
 <CABUDHgLp5sCYrzJG_DLaoypdDvG=zunXjK+uFddmviZRgFWsBg@mail.gmail.com> Happy one month, Julian James!
 <CABUDHgLp5sCYrzJG_DLaoypdDvG=zunXjK+uFddmviZRgFWsBg@mail.gmail.com> Happy one month, Julian James!
 <CAPRuAkYDBeSmUX965NwzLHFQUvU4LmRAc7Zb1ey9vLZSkg4UTw@mail.gmail.com> 
 <CAPRuAkYDBeSmUX965NwzLHFQUvU4LmRAc7Zb1ey9vLZSkg4UTw@mail.gmail.com> 
 <CAPRuAkaM9A_Rhveync6iTyH3NZyF-tuZwpAKn-HGGjnFgMLWpw@mail.gmail.com> None
 <CAPRuAkaM9A_Rhveync6iTyH3NZyF-tuZwpAKn-HGGjnFgMLWpw@mail.gmail.com> None
 <CAPRuAkaM9A_Rhveync6iTyH3NZyF-tuZwpAKn-HGGjnFgMLWpw@mail.gmail.com> None
 <CAPRuAkaM9A_Rhveync6iTyH3NZyF-tuZwpAKn-HGGjnFgMLWpw@mail.gmail.com> None
 <CAPRuA

 <CAPRuAka6KJ-TH2dMyf2zTj3sg4PecJWRt=17oO7vp9SZUxmWtQ@mail.gmail.com> 
 <CAPRuAka6KJ-TH2dMyf2zTj3sg4PecJWRt=17oO7vp9SZUxmWtQ@mail.gmail.com> 
 <CAPRuAka6KJ-TH2dMyf2zTj3sg4PecJWRt=17oO7vp9SZUxmWtQ@mail.gmail.com> 
 <c93bd54e404ae7d3207d342aefb4770c@doctors.org.uk> Re: Nappy
 <CAPRuAkbFCZNHdWrCopkXhikhWOnfam9QuqhtJa8x+mARUbUGRg@mail.gmail.com> pt 5
 <CAPRuAkbFCZNHdWrCopkXhikhWOnfam9QuqhtJa8x+mARUbUGRg@mail.gmail.com> pt 5
 <CAPRuAkbFCZNHdWrCopkXhikhWOnfam9QuqhtJa8x+mARUbUGRg@mail.gmail.com> pt 5
 <CAPRuAkbFCZNHdWrCopkXhikhWOnfam9QuqhtJa8x+mARUbUGRg@mail.gmail.com> pt 5
 <CAPRuAkZA7=vE0YPSka6Xr0kE-3m9XCmiBES6AcPAB4Kkaws-0Q@mail.gmail.com> 
 <CABUDHgLjS9L2p_qgObOf48A26Zbqkft-Y2Diupe190jnYxCeJQ@mail.gmail.com> Fwd: Important: your claim is ready to be processed
 <CABUDHgLjS9L2p_qgObOf48A26Zbqkft-Y2Diupe190jnYxCeJQ@mail.gmail.com> Fwd: Important: your claim is ready to be processed
 <2f4f926c2d3e8ac6edfb4f121360e94e@doctors.org.uk> Re: Vegister postal votes early!
 <2f4f926c2d3e8ac6edfb4f121

 <9880942e9bc0ec7e7e7009e088e18794@doctors.org.uk> Re: Photoshoots
 <9880942e9bc0ec7e7e7009e088e18794@doctors.org.uk> Re: Photoshoots
 <9880942e9bc0ec7e7e7009e088e18794@doctors.org.uk> Re: Photoshoots
 <9880942e9bc0ec7e7e7009e088e18794@doctors.org.uk> Re: Photoshoots
 <9880942e9bc0ec7e7e7009e088e18794@doctors.org.uk> Re: Photoshoots
 <CAPRuAkaEW7wr29DjtBgzXw7a_ceabSoq+r7ekc=du3YYTcW8Vg@mail.gmail.com> Photoshoots
 <CAPRuAkaEW7wr29DjtBgzXw7a_ceabSoq+r7ekc=du3YYTcW8Vg@mail.gmail.com> Photoshoots
 <CAPRuAkaEW7wr29DjtBgzXw7a_ceabSoq+r7ekc=du3YYTcW8Vg@mail.gmail.com> Photoshoots
 <CAPRuAkaEW7wr29DjtBgzXw7a_ceabSoq+r7ekc=du3YYTcW8Vg@mail.gmail.com> Photoshoots
 <CAPRuAkaEW7wr29DjtBgzXw7a_ceabSoq+r7ekc=du3YYTcW8Vg@mail.gmail.com> Photoshoots
 <CAPRuAkbdRWhj8f92cQNFjvLYAQ1SAvu1h1=tM7hvRsntnrykPA@mail.gmail.com> Dan
 <CAF+sTEz7t3JYFid=FQssnQ2FnHByYdHbhuRE3-ZwZER5oVO6qQ@mail.gmail.com> Re: Texas
 <BLUPR0401MB1602F773D02EF611E7EB70FDCFD40@BLUPR0401MB1602.namprd04.prod.outlook.com> Re: Hunmet driv

In [6]:
df = pd.DataFrame(emails)
df.sort_values("timediff")

,_id,email_date,img_date,subject,timediff
221,<0638932e5c78c5a76969048271880819@doctors.org.uk>,2018-07-30 08:46:09+00:00,2018-07-31 00:29:21+00:00,News,-1 days +08:16:48
220,<0638932e5c78c5a76969048271880819@doctors.org.uk>,2018-07-30 08:46:09+00:00,2018-07-31 00:29:21+00:00,News,-1 days +08:16:48
222,<865c6df9255bd5a1229be60e25b229ed@doctors.org.uk>,2018-10-14 17:32:46+00:00,2018-10-15 02:02:48+00:00,Helman Tor,-1 days +15:29:58
223,<865c6df9255bd5a1229be60e25b229ed@doctors.org.uk>,2018-10-14 17:32:46+00:00,2018-10-15 02:02:48+00:00,Helman Tor,-1 days +15:29:58
301,<CAPRuAkag_Kpe_=J1hc43x1sTj2N9b25NHXNBjQxS=huW...,2018-11-11 21:52:00+00:00,2018-11-12 03:47:37+00:00,,-1 days +18:04:23
357,<CAPRuAkbbXm0NajNrCLR76189847EmdgUd1h1O089NxSz...,2013-11-10 15:39:51+00:00,2013-11-10 21:27:52+00:00,Mae and Buddy,-1 days +18:11:59
356,<CAPRuAkbbXm0NajNrCLR76189847EmdgUd1h1O089NxSz...,2013-11-10 15:39:51+00:00,2013-11-10 21:27:52+00:00,Mae and Buddy,-1 days +18:11:59
354,<CAPRuAkbbXm0NajNrCLR76189847EmdgUd1h1O089NxSz...,2013-11-10 15:39:51+00:00,2013-11-10 21:27:52+00:00,Mae and Buddy,-1 days +18:11:59
358,<CAPRuAkbbXm0NajNrCLR76189847EmdgUd1h1O089NxSz...,2013-11-10 15:39:51+00:00,2013-11-10 21:27:52+00:00,Mae and Buddy,-1 days +18:11:59
353,<CAPRuAkbbXm0NajNrCLR76189847EmdgUd1h1O089NxSz...,2013-11-10 15:39:51+00:00,2013-11-10 21:27:52+00:00,Mae and Buddy,-1 days +18:11:59
